## Fake News Classfier Using Bidirectional LSTM

In [ ]:
import pandas as pd


In [ ]:
try:
    df = pd.read_csv('train.csv', on_bad_lines='skip', engine='python')
    print(df.tail())
except pd.errors.ParserError as e:
    print("Error parsing file: ", e)

          id                                              title  \
15129  15114  Donald Trump Says Drugs Are ‘Big Factor’ in Ur...   
15130  15115  Mitch McConnell, Paul Ryan Meet with Donald Tr...   
15131  15116  Susan Rice, Ex-Obama Adviser, Is Back in Polit...   
15132  15117  The Fix Is In: NBC Affiliate Accidentally Post...   
15133  15118  Original unreleased transcript of Sonia Gandhi...   

                    author                                               text  \
15129        Ashley Parker  PITTSBURGH  —   Donald J. Trump said Thursday ...   
15130     Charlie Spiering  House Speaker Paul Ryan and Senate Majority Le...   
15131          Peter Baker  WASHINGTON  —   She has been here before. Afte...   
15132        PatriotRising  Posted 11/03/2016 12:44 am by PatriotRising wi...   
15133  Suresh Subrahmanyan  Original unreleased transcript of Sonia Gandhi...   

       label  
15129    0.0  
15130    0.0  
15131    0.0  
15132    1.0  
15133    1.0  


In [ ]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1.0
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0.0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1.0
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1.0
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1.0


In [ ]:
df.isnull().sum()

id           0
title      403
author    1445
text        48
label       16
dtype: int64

In [ ]:
df.shape

(15134, 5)

In [ ]:
df.dropna()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1.0
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0.0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1.0
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1.0
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1.0
...,...,...,...,...,...
15129,15114,Donald Trump Says Drugs Are ‘Big Factor’ in Ur...,Ashley Parker,PITTSBURGH — Donald J. Trump said Thursday ...,0.0
15130,15115,"Mitch McConnell, Paul Ryan Meet with Donald Tr...",Charlie Spiering,House Speaker Paul Ryan and Senate Majority Le...,0.0
15131,15116,"Susan Rice, Ex-Obama Adviser, Is Back in Polit...",Peter Baker,WASHINGTON — She has been here before. Afte...,0.0
15132,15117,The Fix Is In: NBC Affiliate Accidentally Post...,PatriotRising,Posted 11/03/2016 12:44 am by PatriotRising wi...,1.0


In [ ]:
df.shape

(15134, 5)

In [ ]:
df = df.dropna().reset_index(drop=True)
df = df[pd.to_numeric(df['label'], errors='coerce').notnull()]
df['label']=df['label'].astype(int)

In [ ]:
## get independent features
X=df.drop('label',axis=1)

In [ ]:
y=df['label']

In [ ]:
##check wheather dataset is balanced or not
y.value_counts()

label
0    7545
1    5743
Name: count, dtype: int64

In [ ]:
import tensorflow as tf

In [ ]:
tf.__version__

'2.15.0'

In [ ]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional

In [ ]:
## vocabulary size
voc_size=5000

In [ ]:
message=X.copy()

In [ ]:
message

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
13283,15114,Donald Trump Says Drugs Are ‘Big Factor’ in Ur...,Ashley Parker,PITTSBURGH — Donald J. Trump said Thursday ...
13284,15115,"Mitch McConnell, Paul Ryan Meet with Donald Tr...",Charlie Spiering,House Speaker Paul Ryan and Senate Majority Le...
13285,15116,"Susan Rice, Ex-Obama Adviser, Is Back in Polit...",Peter Baker,WASHINGTON — She has been here before. Afte...
13286,15117,The Fix Is In: NBC Affiliate Accidentally Post...,PatriotRising,Posted 11/03/2016 12:44 am by PatriotRising wi...


In [ ]:
import nltk

In [ ]:
import re
from nltk.corpus import stopwords

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()
corpus=[]
for i in range(0,len(message)):
  review=re.sub('[^a-zA-Z]',' ',message['title'][i])
  review=review.lower()
  review=review.split()

  review=[ps.stem(word) for word in review if not word in stopwords.words('english')]
  review=' '.join(review)
  corpus.append(review)

In [ ]:
len(corpus)

13288

In [ ]:
onehot_repr=[one_hot(words,voc_size) for words in corpus]

In [ ]:
onehot_repr[0]
print(len(onehot_repr))

13288


## Embedding Representation

In [ ]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)

In [ ]:
embedded_docs

array([[   0,    0,    0, ..., 1699, 1560,  542],
       [   0,    0,    0, ..., 2601, 4211, 1013],
       [   0,    0,    0, ..., 3762, 4373, 2549],
       ...,
       [   0,    0,    0, ..., 3008, 1442, 4625],
       [   0,    0,    0, ..., 3268, 1874, 2978],
       [   0,    0,    0, ...,  754, 4287, 3417]], dtype=int32)

In [ ]:
##creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Bidirectional(LSTM(100)))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])
print(model.summary)

<bound method Model.summary of <keras.src.engine.sequential.Sequential object at 0x7db71c5119c0>>


In [ ]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [ ]:
##train test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [ ]:
##model training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=32)

Epoch 1/10
279/279 [==============================] - 12s 29ms/step - loss: 0.2845 - accuracy: 0.8707 - val_loss: 0.1897 - val_accuracy: 0.9234
Epoch 2/10
279/279 [==============================] - 9s 31ms/step - loss: 0.1335 - accuracy: 0.9492 - val_loss: 0.1979 - val_accuracy: 0.9211
Epoch 3/10
279/279 [==============================] - 8s 29ms/step - loss: 0.0815 - accuracy: 0.9697 - val_loss: 0.2338 - val_accuracy: 0.9195
Epoch 4/10
279/279 [==============================] - 7s 25ms/step - loss: 0.0441 - accuracy: 0.9860 - val_loss: 0.3218 - val_accuracy: 0.9102
Epoch 5/10
279/279 [==============================] - 8s 30ms/step - loss: 0.0281 - accuracy: 0.9909 - val_loss: 0.3274 - val_accuracy: 0.9191
Epoch 6/10
279/279 [==============================] - 8s 29ms/step - loss: 0.0157 - accuracy: 0.9952 - val_loss: 0.5188 - val_accuracy: 0.9104
Epoch 7/10
279/279 [==============================] - 9s 31ms/step - loss: 0.0113 - accuracy: 0.9966 - val_loss: 0.5063 - val_accuracy: 0.906

In [ ]:
y_pred=model.predict(X_test)

138/138 [==============================] - 2s 7ms/step


In [ ]:
y_pred

array([[1.0117904e-04],
       [1.0243732e-05],
       [2.3539160e-09],
       ...,
       [1.7412756e-05],
       [9.9995267e-01],
       [5.7963669e-05]], dtype=float32)

In [ ]:
import numpy as np

y_pred>=0.5

array([[False],
       [False],
       [False],
       ...,
       [False],
       [ True],
       [False]])

In [ ]:
y_pred=np.where(y_pred>=0.5,1,0)

In [ ]:
from sklearn.metrics import confusion_matrix,accuracy_score
confusion_matrix(y_test,y_pred)

array([[2282,  220],
       [ 200, 1684]])

In [ ]:
print(accuracy_score(y_test,y_pred))

0.9042407660738714
